In [22]:
import nest_asyncio
nest_asyncio.apply()

In [15]:
import getpass
import os

if not os.environ.get("TAVILY_API_KEY"):
    os.environ["TAVILY_API_KEY"] = getpass.getpass("Tavily API key:\n")

In [4]:
from langchain_tavily import TavilySearch

tool = TavilySearch(
    max_results=1,
    topic="general",
)

In [7]:
result = tool.invoke({"query": "What happened at the last wimbledon"})
print(result['results'])

[{'title': "Andy Murray pulls out of the men's singles draw at his last Wimbledon", 'url': 'https://www.nbcnews.com/news/sports/andy-murray-wimbledon-tennis-singles-draw-rcna159912', 'content': "NBC News Now LONDON — Andy Murray, one of the last decade's most successful male tennis players, has pulled out of the singles tournament at what is almost certain to be his last Wimbledon, his team confirmed Tuesday. Murray, 37, who has won the Wimbledon singles title twice and the U.S Open once, has been battling to be fit to play at the All England Club for weeks. “Unfortunately, despite working incredibly hard on his recovery since his operation just over a week ago, Andy has taken the very difficult decision not to play the singles this year,” his team said in a statement reported by Sky News. The news caps a glittering career on the ATP singles tour, which placed Murray at No. 1 in the world for 41 weeks.", 'score': 0.6755297, 'raw_content': None}]


In [9]:
# This is usually generated by a model, but we'll create a tool call directly for demo purposes.
model_generated_tool_call = {
    "args": {"query": "euro 2024 host nation"},
    "id": "1",
    "name": "tavily",
    "type": "tool_call",
}
tool_msg = tool.invoke(model_generated_tool_call)

# The content is a JSON string of results
print(tool_msg)

content='{"query": "euro 2024 host nation", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "UEFA Euro 2024 - Wikipedia", "url": "https://en.wikipedia.org/wiki/UEFA_Euro_2024", "content": "Tournament details Host country Germany Dates 14 June – 14 July Teams 24 Venue(s) 10 (in 10 host cities) Final positions Champions Spain (4th title) Runners-up England Tournament statistics Matches played 51 Goals scored 117 (2.29 per match) Attendance 2,681,288 (52,574 per match) Top scorer(s) Harry Kane Georges Mikautadze Jamal Musiala Cody Gakpo Ivan Schranz Dani Olmo (3 goals each) Best player(s) Rodri Best young player Lamine Yamal ← 2020 2028 → The 2024 UEFA European Football Championship, commonly referred to as UEFA Euro 2024 (stylised as UEFA EURO 2024) or simply Euro 2024, was the 17th UEFA European Championship, the quadrennial international football championship organised by UEFA for the European men\'s national teams of their member associations. Germany 

In [20]:
from agents import Agent, FunctionTool, RunContextWrapper,Runner, function_tool
from langchain_tavily import TavilySearch
from typing_extensions import TypedDict, Any

In [34]:
@function_tool(name_override="research")
async def search_internet(ctx: RunContextWrapper[Any], query: str) -> str:
    """Search the internet and gather the important information.

    Args:
        query: The query to search for.
    """
    tool = TavilySearch(
    max_results=5,
    topic="general",
)
    return tool.invoke({"query": query})

In [35]:
from dotenv import load_dotenv
from agents import OpenAIChatCompletionsModel, AsyncOpenAI
from agents.run import RunConfig

In [36]:
gemini_api_key = os.getenv("GEMINI_API_KEY")

    # Check if the API key is present; if not, raise an error
if not gemini_api_key:
        raise ValueError(
            "GEMINI_API_KEY is not set. Please ensure it is defined in your .env file.")

    # Reference: https://ai.google.dev/gemini-api/docs/openai
client =  AsyncOpenAI(
        api_key=gemini_api_key,
        base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
    )

In [37]:
def create_model(client, model_name="gemini-2.0-flash"):
    """Create a model instance with the specified client and model name"""

    return OpenAIChatCompletionsModel(
        model=model_name,
        openai_client=client
    )

In [38]:
def create_run_config(model, client):
    """Create a run configuration for agents"""
    return RunConfig(
        model=model,
        model_provider=client,
        tracing_disabled=True
    )

model = create_model(client=client)
config  = create_run_config(model, client)

In [39]:
agent = Agent(
    name="You are a Market & Audience Research Agent who can conduct a comprehensive market and competitor analysis. You can audit the current brand presence and sentiment. You can develop detailed audience personas by gathering demographic, psychographic, and behavioral data.",
    tools=[search_internet],  
)

In [42]:
result = Runner.run_sync(agent,input="How many followers does Scents n Stories have on social media platforms like facebook and instagram?",run_config=config)

print("\nCALLING AGENT\n")
print(result.final_output)


CALLING AGENT

Scents n Stories has multiple Facebook and Instagram accounts. Here's a breakdown of their followers:

**Instagram:**

*   @scentsnstories: 204K followers
*   @usa\_scentsnstories: 141 followers

**Facebook:**

*   Scents N Stories: 130,482 likes
*   Scents N Stories - International: 46,107 likes
*   Scents N Stories USA: (No follower count available)
